# 3.5 Streams

In [1]:
; define as macros to override Schemes eager evalutation
(define-syntax delay
               [(delay ?exp) (memo-proc (lambda() ?exp))])
(define-syntax cons-stream
               [(cons-stream ?a ?b) (cons ?a (delay ?b))])

(define the-empty-stream '())

(define (stream-null? s)
  (eq? s the-empty-stream))
(define (stream-car stream)
  (car stream))
(define (stream-cdr stream)
  (force (cdr stream)))

(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))

(define (stream-map proc s)
  (if (stream-null? s)
      the-empty-stream
      (cons-stram
       (proc (stream-car s))
       (stream-map proc (stream-cdr s)))))

(define (stream-for-each proc s)
  (if (stream-null? s)
      'done
      (begin
       (proc (stream-car s))
       (stream-for-each proc
                        (stream-cdr s)))))

(define (stream-filter pred stream)
  (cond ((stream-null? stream)
         the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream
          (stream-car stream)
          (stream-filter
           pred
           (stream-cdr stream))))
        (else (stream-filter
               pred
               (stream-cdr stream)))))

(define (stream-enumerate-interval low high)
  (if (> low high)
      the-empty-stream
      (cons-stream
       low
       (stream-enumerate-interval (+ low 1)
                                  high))))

(define (force delayed-object)
  (delayed-object))

(define (memo-proc proc)
  (let ((already-run? #f)
        (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))

(define (display-stream s)
  (stream-for-each display-line s))

(define (display-line x)
  (newline)
  (display x))

## 3.50

In [2]:
; take proc of n args along with n streams
; apply proc to to all elements of each stream
(define (stream-map proc . argstreams)
  (if (stream-null? (car argstreams))
      the-empty-stream
      (cons-stream
       (apply proc (map stream-car argstreams))
       (apply stream-map
              (cons proc
                    (map stream-cdr
                         argstreams))))))

## 3.51

In [3]:
; prints and returns argument
(define (show x)
  (display-line x)
  x)

(define x
  (stream-map
   show
   (stream-enumerate-interval 0 10)))


0

In [4]:
(stream-ref x 5)


1
2
3
4
5

5

In [5]:
(stream-ref x 7)


6
7

7

## 3.52

In [6]:
(define sum 0)

sum

0

In [7]:
(define (accum x)
  (set! sum (+ x sum))
  sum)

sum

0

In [8]:
(define seq
  (stream-map
   accum
   (stream-enumerate-interval 1 20)))

sum

1

In [9]:
(define y (stream-filter even? seq))

sum

6

In [10]:
(define z
  (stream-filter
   (lambda (x)
     (= (remainder x 5) 0)) seq))

sum

10

In [11]:
(stream-ref y 7)

136

In [12]:
sum

136

In [13]:
(display-stream z)


10
15
45
55
105
120
190
210

done

In [14]:
sum

210

# 3.5.2 Infinite Streams

In [16]:
(define ones (cons-stream 1 ones))

(define (add-streams s1 s2)
  (stream-map + s1 s2))

(define integers
  (cons-stream 1 (add-streams ones integers)))

(define fibs
  (cons-stream
   0 (cons-stream
      1 (add-streams
         (stream-cdr fibs) fibs))))

(define (scale-stream stream factor)
  (stream-map
   (lambda (x) (* x factor))
   stream))

(define primes
  (cons-stream
   2 (stream-filter
      prime? (integers-starting-from 3))))

; check whether n is divisible by a prime less than or equal to sqrt(n)
(define (prime? n)
  (define (iter ps)
    (cond ((> (square (stream-car ps)) n) #t)
          ((divisible? n (stream-car ps)) #f)
          (else (iter (stream-cdr ps)))))
  (iter primes))

## 3.53

```scheme
(define s (cons-stream 1 (add-streams s s)))
```
`s` is an infinite stream of powers of 2: `1, 2, 4, 8, 16, 32 ...`

It is equivalent to the `double` stream defined earlier in the chapter:
```scheme
(define double
  (cons-stream 1 (scale-stream double 2)))
```

## 3.54

In [17]:
(define (mul-streams s1 s2)
  (stream-map * s1 s2))

(define factorials
  (cons-stream 1 (mul-streams factorials integers)))

## 3.55

In [18]:
(define (partial-sums stream)
   (cons-stream (stream-car s)
                (add-streams (stream-cdr s)
                             (partial-sum s))))

## 3.56

In [19]:
(define (merge s1 s2)
  (cond ((stream-null? s1) s2)
        ((stream-null? s2) s1)
        (else
         (let ((s1car (stream-car s1))
               (s2car (stream-car s2)))
           (cond ((< s1car s2car)
                  (cons-stream
                   s1car
                   (merge (stream-cdr s1)
                          s2)))
                 ((> s1car s2car)
                  (cons-stream
                   s2car
                   (merge s1
                          (stream-cdr s2))))
                 (else
                  (cons-stream
                   s1car
                   (merge
                    (stream-cdr s1)
                    (stream-cdr s2)))))))))

(define S (cons-stream
           1
           (merge (scale-stream S 2) (merge (scale-stream S 3)
                                            (scale-stream S 5)))))

## 3.57
```scheme
(define fibs
  (cons-stream
   0 (cons-stream
      1 (add-streams
         (stream-cdr fibs) fibs))))
```
The streams definition of `fibs` will perform $n-1$ additions when computing the $n^{th}$ Fibonacci number.

Each term $Fib _n$ is computed by $sum(Fib _{n-1}, Fib_{n-2})$. Due to memoization in `delay`, recalling the previous terms $Fib _{n-1}, Fib_{n-2}$ requires no additions. Therefore, each term *after the first* requires **one addition** (the first term is given in the definition of `fibs`).

Without memoization `delay`, computing $Fib _n$ will require re-calculating $Fib _{n-1}$ and $Fib_{n-2}$ instead of recalling the memoized values:

$Fib _1 = 1$

$T(Fib _{n-1}) = O(2^{n-1})$

$T(Fib _n) = T(Fib _n-1) + T(Fib _n-2) = O(2^{n-1}) + O(2^{n-2}) = O(2^n)$


## 3.58

In [20]:
(define (expand num den radix)
  (cons-stream
   (quotient (* num radix) den)
   (expand (remainder (* num radix) den)
           den
           radix)))

; print first n values of stream 
(define (display-stream-n s n)
  (define (iter i)
    (if (< i n)
        (begin 
         (display (stream-ref s i))
         (iter (+ i 1)))))
  (iter 0))

`expand` produces the digits of $\frac{num}{den}$ in base `radix`:

In [21]:
(display-stream-n (expand 1 7 10) 5)

14285

#f

In [22]:
(float (/ 1 7))

0.14285714285714285

In [23]:
(display-stream-n (expand 3 8 10) 5)

37500

#f

In [24]:
(float (/ 3 8))

0.375

## 3.59

In [29]:
(define (integrate-series s)
  (stream-map / s integers))

(define exp-series
  (cons-stream
   1 (integrate-series exp-series)))

(define cosine-series
  (cons-stream 1 (integrate-series sine-series)))

(define sine-series
  (cons-stream
   0 (scale-stream (integrate-series cosine-series) -1)))